# *ML-7. Прогнозирование биологического ответа (HW-3)*

#### *Необходимо*:
1. обучить две модели: 
    - логистическую регрессию
    - случайный лес. 
2. сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. 
Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.

*В качестве метрики будем использовать F1-score.*



In [1]:
# Импортируем используемые библиотеки.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model
from sklearn import ensemble
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
import optuna


In [2]:
# Загружаем датафрейм.
bio_answer = pd.read_csv('data/_train_sem09 (1).csv')
bio_answer

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3746,1,0.033300,0.506409,0.10,0.0,0.209887,0.633426,0.297659,0.376124,0.727093,...,0,0,0,0,0,0,0,0,0,0
3747,1,0.133333,0.651023,0.15,0.0,0.151154,0.766505,0.170876,0.404546,0.787935,...,0,0,1,0,1,0,1,0,0,0
3748,0,0.200000,0.520564,0.00,0.0,0.179949,0.768785,0.177341,0.471179,0.872241,...,0,0,0,0,0,0,0,0,0,0
3749,1,0.100000,0.765646,0.00,0.0,0.536954,0.634936,0.342713,0.447162,0.672689,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# Отделяем из датасет целевой признак.
y = bio_answer['Activity']
x = bio_answer.drop(['Activity'], axis=1)

# Разбиваем данные на тренировочную и тестовую.
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1, stratify=y)

## *1. Логистическая регрессия*

#### Посмотрим на метрику f1-score при устрановленных по умолчанию гиперпараметров.

In [4]:
# Обучаем модель.
log_reg = linear_model.LogisticRegression(random_state=42, max_iter=600)
log_reg.fit(X_train, y_train)

# Делаем предсказание.
log_reg_train_pred = log_reg.predict(X_train)
log_reg_test_pred = log_reg.predict(X_test)

# Рассчитываем метрики.
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, log_reg_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(log_reg.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, log_reg_test_pred)))

f1_score на тренировочном наборе: 0.90
accuracy на тестовом наборе: 0.76
f1_score на тестовом наборе: 0.78


### *1.1 GridSearchCV*

In [5]:
# Задаем искомые гиперпараметры в виде словаря.
param_grid = {'penalty': ['l2'],
              'solver': ['lbfgs', 'sag'],
              'C': list(np.linspace(0.1, 0.5, 20, dtype=float))
              }

# С помощью GridSearchCV находим наилучшие параметры для логистической регрессии.
grid_search_lr = GridSearchCV(
    estimator=linear_model.LogisticRegression(
    random_state=42,
    max_iter=1000
    ), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  

# Обучаем модель и предсказываем результаты.
%time grid_search_lr.fit(X_train, y_train)
y_train_pred = grid_search_lr.predict(X_train)
y_test_pred = grid_search_lr.predict(X_test)

# Рассчитываем метрики.
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(grid_search_lr.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_lr.best_params_))

CPU times: user 1.37 s, sys: 213 ms, total: 1.58 s
Wall time: 9min 50s
f1_score на тренировочном наборе: 0.85
accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.80
Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}


_________________
### *Вывод:*
*С помощью подобранных гиперпараметров классом GridSearchCV нам удалось улучшить метрику f1-score, она составила - 0.80.*

_________________

###  ***1.2 RandomizedSearchCV***

In [7]:
# Задаем искомые гиперпараметры в виде словаря.
param_distributions = {'penalty': ['l2'] ,
              'solver': ['lbfgs', 'sag'],
               'C': list(np.linspace(0.1, 0.5, 20, dtype=float))},

# С помощью RandomizedSearchCV находим наилучшие параметры для логистической регрессии.            
random_search_lr = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=1000), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 15, 
    n_jobs = -1
)  

# Обучаем модель и предсказываем результаты.
%time random_search_lr.fit(X_train, y_train)
y_train_pred = random_search_lr.predict(X_train)
y_test_pred = random_search_lr.predict(X_test)

# Рассчитываем метрики.
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(random_search_lr.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_lr.best_params_))

CPU times: user 1.12 s, sys: 144 ms, total: 1.26 s
Wall time: 2min 51s
f1_score на тренировочном наборе: 0.85
accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.80
Наилучшие значения гиперпараметров: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.1}


_________________
### *Вывод:*
*С помощью подобранных гиперпараметров классом RandomizedSearchCV нашли такие же гиперпараметры как и при GridSearchCV. Но при этом время для их поиска намного сократилось в 3 раза.*

_________________

###  *1.3 Hyperopt*

In [8]:
# Задаем искомые гиперпараметры в виде словаря.
penalties = ['l2']
solvers = ['sag', 'lbfgs']
space = {'penalty': hp.choice('penalty', penalties),
        'solver': hp.choice('solver', solvers),
        'C': hp.uniform('C',0,1)}


def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=42):
    """Функция получения комбинации гиперпараметров."""

    # Строим модель.
    model = linear_model.LogisticRegression(**params, random_state=42, max_iter=5000)

    # Обучаем модель с помощью кросс-валидации
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # Метрику необходимо минимизировать, поэтому ставим знак минус.
    return -score


# Залогируем полученные результаты.
trials = Trials() # используется для логирования результатов

# Начинаем подбор гиперпараметров.
best = fmin(hyperopt_rf,
          space=space,
          algo=tpe.suggest,
          max_evals=25,
          trials=trials,
          rstate=np.random.default_rng(42)
         )

#Выводим наилучшие значения гиперпараметров.
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 25/25 [06:57<00:00, 16.68s/trial, best loss: -0.7785419212939304]
Наилучшие значения гиперпараметров {'C': 0.09664464567700914, 'penalty': 0, 'solver': 0}


In [11]:
# Строим логистическую модель.
model = linear_model.LogisticRegression(
    max_iter=1000,
    random_state=42, 
    penalty=penalties[best['penalty']],
    solver=solvers[best['solver']],
    C=best['C']
)

# Обучаем модель.
model.fit(X_train, y_train)

# Предсказываем результат для тренировочной и тестовой выборки.
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Рассчитываем метрики.
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 0.85
accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.80


In [13]:
# Продолжим подбор наилучших гиперпараметров.
best = fmin(hyperopt_rf,
          space=space,
          algo=tpe.suggest,
          max_evals=30,
          trials=trials,
          rstate=np.random.default_rng(42)
         )

#Выводим наилучшие значения гиперпараметров.
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 30/30 [00:00<?, ?trial/s, best loss=?]
Наилучшие значения гиперпараметров {'C': 0.09664464567700914, 'penalty': 0, 'solver': 0}


In [14]:
# Строим еще раз логистическую модель.
model = linear_model.LogisticRegression(
    max_iter=1000,
    random_state=42, 
    penalty=penalties[best['penalty']],
    solver=solvers[best['solver']],
    C=best['C']
)

# Обучаем модель.
model.fit(X_train, y_train)

# Предсказываем результат для тренировочной и тестовой выборки.
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Рассчитываем метрики.
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 0.85
accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.80


_________________
### *Вывод:*
*С помощью Hyperopt и использования кросс-валидации метрика f1-score составила - 0.80. Нам неудалось еще больше улучшить результаты.*

_________________

###  *1.4 Optuna*

In [15]:
def optuna_rf(trial, cv=5, X=X_train, y=y_train, random_state=42):
    """Функция получения комбинации гиперпараметров."""
    
    # Задаем пространства поиска гиперпараметров.
    penalties = ['l2']
    solvers = ['sag', 'lbfgs']
    penalty = trial.suggest_categorical('penalty', penalties)
    solver = trial.suggest_categorical('solver', solvers)
    C = trial.suggest_float('C', 0, 1)

    # Создаем модель.
    model = linear_model.LogisticRegression(penalty=penalty,
                                          solver=solver,
                                          C=C,
                                          random_state=42,
                                          max_iter=1000)
    # Обучаем модель.
    model.fit(X_train, y_train)
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    return score


# Создаем объект исследования.
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")

# Ищем лучшую комбинацию гиперпараметров n_trials раз.
study.optimize(optuna_rf, n_trials=20)

print("Наилучшие значения гиперпараметров {}".format(study.best_params))


[I 2023-04-02 15:43:31,519] A new study created in memory with name: LogisticRegression
[I 2023-04-02 15:43:37,646] Trial 0 finished with value: 0.7702292241071234 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.26661651175110335}. Best is trial 0 with value: 0.7702292241071234.
[I 2023-04-02 15:43:44,813] Trial 1 finished with value: 0.7732374671257205 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.864145007132271}. Best is trial 1 with value: 0.7732374671257205.
[I 2023-04-02 15:44:23,294] Trial 2 finished with value: 0.7767224176907688 and parameters: {'penalty': 'l2', 'solver': 'sag', 'C': 0.13326669366771804}. Best is trial 2 with value: 0.7767224176907688.
[I 2023-04-02 15:45:06,240] Trial 3 finished with value: 0.7773132138958562 and parameters: {'penalty': 'l2', 'solver': 'sag', 'C': 0.1377743297529077}. Best is trial 3 with value: 0.7773132138958562.
[I 2023-04-02 15:45:46,858] Trial 4 finished with value: 0.7741039366297284 and parameters: {'penalty':

Наилучшие значения гиперпараметров {'penalty': 'l2', 'solver': 'sag', 'C': 0.023369730276901035}


In [16]:
# Создаем модель.
model = linear_model.LogisticRegression(**study.best_params,random_state=42, max_iter=1000)

# Обучаем модель.
model.fit(X_train, y_train)

# Предсказываем результат для тренировочной и тестовой выборки.
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Рассчитываем метрики.
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 0.83
accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.80


_________________
### *Вывод:*
*С помощью гиперпараметров найденных Optuna получили такие же значения метрики f1-score как и при других классах 0.80.*

_________________

## *2. Случайный лес*

#### Посмотрим на метрику f1-score при устрановленных по умолчанию гиперпараметров.

In [17]:
# Обучаем модель.
forest = ensemble.RandomForestClassifier(random_state=42)
forest.fit(X_train, y_train)

# Делаем предсказание.
forest_train_pred = forest.predict(X_train)
forest_test_pred = forest.predict(X_test)

# Рассчитываем метрики.
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, forest_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(forest.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, forest_test_pred)))

f1_score на тренировочном наборе: 1.00
accuracy на тестовом наборе: 0.80
f1_score на тестовом наборе: 0.82


### *2.1 GridSearchCV*

In [18]:
# Задаем искомые гиперпараметры в виде словаря.
param_grid = {'n_estimators': list(range(80, 200, 30)),
            'min_samples_leaf': [3],
            'max_depth': list(np.linspace(20, 40, 5, dtype=int)),
            'criterion': ['gini', 'entropy']
            }

# С помощью GridSearchCV находим наилучшие параметры для логистической регрессии.
grid_search_forest = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  

# Обучаем модель и предсказываем результаты.
%time grid_search_forest.fit(X_train, y_train)
y_train_pred = grid_search_forest.predict(X_train)
y_test_pred = grid_search_forest.predict(X_test)

# Рассчитываем метрики.
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(grid_search_forest.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_forest.best_params_))

CPU times: user 1.68 s, sys: 103 ms, total: 1.78 s
Wall time: 1min 45s
f1_score на тренировочном наборе: 0.98
accuracy на тестовом наборе: 0.79
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'criterion': 'entropy', 'max_depth': 30, 'min_samples_leaf': 3, 'n_estimators': 110}


_________________
### *Вывод:*
*С помощью подобранных гипераметров классом GridSearchCV нам не удалось улучшить метрику f1-score и она также составила 0.82.*

_________________

###  *2.2 RandomizedSearchCV*

In [19]:
# Задаем искомые гиперпараметры в виде словаря.
param_distributions = {'n_estimators': list(range(80, 200, 30)),
            'min_samples_leaf': [3],
            'max_depth': list(np.linspace(20, 40, 5, dtype=int)),
            'criterion': ['gini', 'entropy']
            }

# С помощью RandomizedSearchCV находим наилучшие параметры для логистической регрессии.            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 15, 
    n_jobs = -1
)  

# Обучаем модель и предсказываем результаты.
%time random_search_forest.fit(X_train, y_train)
y_train_pred = random_search_forest.predict(X_train)
y_test_pred = random_search_forest.predict(X_test)

# Рассчитываем метрики.
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(random_search_forest.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

CPU times: user 1.43 s, sys: 64.7 ms, total: 1.49 s
Wall time: 40.5 s
f1_score на тренировочном наборе: 0.98
accuracy на тестовом наборе: 0.79
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'n_estimators': 110, 'min_samples_leaf': 3, 'max_depth': 30, 'criterion': 'entropy'}


_________________
### *Вывод:*
*С помощью подобранных гиперпараметров классом RandomizedSearchCV метрика f1-score осталась такая же как и при помощи  GridSearchCV.*

_________________

###  *2.3 Hyperopt*

In [20]:
# Задаем искомые гиперпараметры в виде словаря.
criterions = ['gini', 'entropy']
space={'n_estimators': hp.quniform('n_estimators', 80, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 40, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1),
       'criterion': hp.choice('criterion', criterions)
      }


def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=42):
    """Функция получения комбинации гиперпараметров."""
    params = {'n_estimators': int(params['n_estimators']), 
            'max_depth': int(params['max_depth']), 
            'min_samples_leaf': int(params['min_samples_leaf']),
            'criterion': params['criterion']
            }
    # Строим модель.
    model = ensemble.RandomForestClassifier(**params, random_state=42)

    # Обучаем модель с помощью кросс-валидации
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # Метрику необходимо минимизировать, поэтому ставим знак минус.
    return -score


# Залогируем полученные результаты.
trials = Trials() # используется для логирования результатов

# Начинаем подбор гиперпараметров.
best = fmin(hyperopt_rf,
          space=space,
          algo=tpe.suggest,
          max_evals=25,
          trials=trials,
          rstate=np.random.default_rng(42)
         )

#Выводим наилучшие значения гиперпараметров.
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 25/25 [01:29<00:00,  3.60s/trial, best loss: -0.8127261258298031]
Наилучшие значения гиперпараметров {'criterion': 1, 'max_depth': 22.0, 'min_samples_leaf': 2.0, 'n_estimators': 154.0}


In [21]:
# Строим  модель.
model = ensemble.RandomForestClassifier(
    random_state=42, 
    n_estimators=int(best['n_estimators']),
    criterion=criterions[best['criterion']],
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)

# Обучаем модель.
model.fit(X_train, y_train)

# Предсказываем результат для тренировочной и тестовой выборки.
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Рассчитываем метрики.
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 1.00
accuracy на тестовом наборе: 0.80
f1_score на тестовом наборе: 0.82


In [22]:
# Продолжим подбор наилучших гиперпараметров.
best = fmin(hyperopt_rf,
          space=space,
          algo=tpe.suggest,
          max_evals=30,
          trials=trials,
          rstate=np.random.default_rng(42)
         )

#Выводим наилучшие значения гиперпараметров.
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 30/30 [00:23<00:00,  4.65s/trial, best loss: -0.8127261258298031]
Наилучшие значения гиперпараметров {'criterion': 1, 'max_depth': 22.0, 'min_samples_leaf': 2.0, 'n_estimators': 154.0}


In [23]:
# Строим модель.
model = ensemble.RandomForestClassifier(
    random_state=42, 
    n_estimators=int(best['n_estimators']),
    criterion=criterions[best['criterion']],
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)

# Обучаем модель.
model.fit(X_train, y_train)

# Предсказываем результат для тренировочной и тестовой выборки.
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Рассчитываем метрики.
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 1.00
accuracy на тестовом наборе: 0.80
f1_score на тестовом наборе: 0.82


_________________
### *Вывод:*
*С помощью Hyperopt мы не смогли преулучшить f1-score.*

_________________

###  *2.4 Optuna*

In [24]:

def optuna_rf(trial, cv=5, X=X_train, y=y_train, random_state=42):
    """Функция получения комбинации гиперпараметров."""
    
    # Задаем пространства поиска гиперпараметров.
    criterions = ['gini', 'entropy']
    n_estimators = trial.suggest_int('n_estimators', 50, 200, 1)
    max_depth = trial.suggest_int('max_depth', 5, 40, 1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)
    criterion = trial.suggest_categorical('criterion', criterions)

    # Создаем модель.
    model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                        max_depth=max_depth,
                                        min_samples_leaf=min_samples_leaf,
                                        criterion=criterion,
                                        random_state=42)
    # Обучаем модель.
    model.fit(X_train, y_train)
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    return score


# Создаем объект исследования.
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")

# Ищем лучшую комбинацию гиперпараметров n_trials раз.
study.optimize(optuna_rf, n_trials=30)

print("Наилучшие значения гиперпараметров {}".format(study.best_params))


[I 2023-04-02 16:03:55,466] A new study created in memory with name: RandomForestClassifier
[I 2023-04-02 16:04:00,663] Trial 0 finished with value: 0.7940239110732945 and parameters: {'n_estimators': 135, 'max_depth': 25, 'min_samples_leaf': 6, 'criterion': 'entropy'}. Best is trial 0 with value: 0.7940239110732945.
[I 2023-04-02 16:04:03,804] Trial 1 finished with value: 0.7915754690655279 and parameters: {'n_estimators': 93, 'max_depth': 35, 'min_samples_leaf': 8, 'criterion': 'entropy'}. Best is trial 0 with value: 0.7940239110732945.
[I 2023-04-02 16:04:07,367] Trial 2 finished with value: 0.792933734704751 and parameters: {'n_estimators': 103, 'max_depth': 32, 'min_samples_leaf': 7, 'criterion': 'entropy'}. Best is trial 0 with value: 0.7940239110732945.
[I 2023-04-02 16:04:09,541] Trial 3 finished with value: 0.7960092799896223 and parameters: {'n_estimators': 70, 'max_depth': 37, 'min_samples_leaf': 10, 'criterion': 'gini'}. Best is trial 3 with value: 0.7960092799896223.
[I 20

Наилучшие значения гиперпараметров {'n_estimators': 141, 'max_depth': 30, 'min_samples_leaf': 2, 'criterion': 'gini'}


In [25]:
# Создаем модель.
model = ensemble.RandomForestClassifier(**study.best_params,random_state=42)

# Обучаем модель.
model.fit(X_train, y_train)

# Предсказываем результат для тренировочной и тестовой выборки.
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Рассчитываем метрики.
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 0.99
accuracy на тестовом наборе: 0.80
f1_score на тестовом наборе: 0.82


_________________
### *Вывод:*
*С помощью Optuna не удалось еще больше учучшить f1-score.*

*Наилучший f1-score мы получили при использовании модели случайного леса и она составила 0.82. При логистической регресии она составила 0.80. Все подобранные разными методами гиперпараметры не смогли улучшить метрику.*
_________________